In [2]:
import get_started_doing_something_variables
import get_stopped_doing_something_variables
import get_all_vars

import pandas as pd
import read_a_combination_of_variables
import os
import sys
sys.path.insert(0, os.path.abspath('../'))
from helpers.s3_bucket_utils import S3BucketUtils
from helpers import settings
import ast

bucket = S3BucketUtils()

def get_key_based_on_value_in_a_dict(dict_, value_of_interest):
    for key, value in dict_.items():
        if value == value_of_interest:
            return key

def get_model_number(var_, model_names):
    if len(model_names[model_names['model_name']==var_]['model_number'])>0:
        model_number = model_names[model_names['model_name']==var_]['model_number'].iloc[0]
    else:
        multiple_vars = model_names[(model_names['model_name'].apply(lambda x: ',' in x))]
        for i in range(0, len(multiple_vars)):
            if var_ in multiple_vars['model_name'].iloc[i]:
                model_number = multiple_vars['model_number'].iloc[i]
                break
    return model_number

def get_pairs_of_variables(churn_based_on_behaviour_dir, date_dir, model_type):
    vars_periods_to_look_at_thresholds_to_use = \
    bucket.load_csv_from_s3(file_name = churn_based_on_behaviour_dir + 'data/' + date_dir +\
                     '/vars_periods_to_look_at_thresholds_to_use.csv')

    period_to_look_at =\
    vars_periods_to_look_at_thresholds_to_use[['variable_base_name', 'period_to_look_at', 'number_of_months']].\
    apply(lambda x: (x['variable_base_name'], x['period_to_look_at']), axis=1).tolist()


    (all_vars, all_vars_base_names) = get_all_vars.main()
    first_var = dict.fromkeys(all_vars_base_names.values())
    second_var = dict.fromkeys(all_vars_base_names.values())

    if model_type == 'started_doing_something':
        for i in range(0, len(period_to_look_at)):
            first_var[period_to_look_at[i][0]] = 'had_'+period_to_look_at[i][0]+'_'+period_to_look_at[i][1]
            second_var[period_to_look_at[i][0]] = \
            'had_'+period_to_look_at[i][0]+'_before_and_didnt_'+period_to_look_at[i][1]
    elif model_type == 'stopped_doing_something':
        for i in range(0, len(period_to_look_at)):
            first_var[period_to_look_at[i][0]] = 'had_'+period_to_look_at[i][0]+'_before'
            second_var[period_to_look_at[i][0]] = 'had_'+period_to_look_at[i][0]+'_'+period_to_look_at[i][1]

    return (first_var, second_var)

date_of_analysis = input('date of analysis: ')
date_dir = date_of_analysis.replace('-', '_')

model_type = input('Model type: ')

old_models_path = 'combinations_and_names_Dec_10th/'
curr_models_path = 'combinations_and_names_Dec_21st/'

var_ = 'had_online_orders_last_month'

vars_diff = \
pd.read_csv('data/'+date_dir+'/exports/'+model_type+'/not_among_newest_important_variables.csv')
variables_not_in_the_current_combination = dict.fromkeys(vars_diff['variable'])
(first_var, second_var) = \
get_pairs_of_variables(churn_based_on_behaviour_dir='churn_analysis_based_on_behaviour/', date_dir=date_dir,\
                       model_type=model_type)

if var_ in first_var.values():
    base_var_ = get_key_based_on_value_in_a_dict(dict_=first_var, value_of_interest=var_)
elif var_ in second_var.values():
    base_var_ = get_key_based_on_value_in_a_dict(dict_=second_var, value_of_interest=var_)
else:
    base_var_ = var_

old_model_names = \
pd.read_csv('combinations_of_variables_that_are_not_dependent/'+old_models_path+'model_names.csv')
old_combinations = \
pd.read_csv('combinations_of_variables_that_are_not_dependent/'+old_models_path+'combinations_of_variables.csv')
models_with_p_below_0_2 = []
models_with_p_below_0_2_df = pd.DataFrame(columns = ['model_number', 'model_name', 'data_set', 'variable', 'coef', 'exp(coef)', 'p'])
for model_num in old_model_names['model_number'][1:]:
    files = \
    os.listdir('data/'+date_dir+'/exports/coefficients_and_pvalues/'+model_type+'/models_with_old_combinations/model_'+str(model_num)+'/')

    for i in range(0, len(files)):
        old_coefs_and_pvalues = \
        pd.read_csv('data/'+date_dir+'/exports/coefficients_and_pvalues/'+model_type+'/models_with_old_combinations/model_'+str(model_num)+'/'+\
           files[i])
        if len(old_coefs_and_pvalues[(old_coefs_and_pvalues['covariate']==var_)&\
                     (old_coefs_and_pvalues['p']<0.2)]):
            models_with_p_below_0_2.append((model_num, files[i]))
            idx = len(models_with_p_below_0_2_df)
            models_with_p_below_0_2_df.loc[idx, 'model_number'] = model_num
            models_with_p_below_0_2_df.loc[idx, 'model_name'] = str(old_model_names[old_model_names['model_number']==model_num]['model_name'].iloc[0])
            models_with_p_below_0_2_df.loc[idx, 'data_set'] = files[i].split('.csv')[0]
            models_with_p_below_0_2_df.loc[idx, 'variable'] = var_
            models_with_p_below_0_2_df.loc[idx, 'coef'] = \
            old_coefs_and_pvalues[old_coefs_and_pvalues['covariate']==var_]['coef'].iloc[0]
            models_with_p_below_0_2_df.loc[idx, 'exp(coef)'] =\
            old_coefs_and_pvalues[old_coefs_and_pvalues['covariate']==var_]['exp(coef)'].iloc[0]
            models_with_p_below_0_2_df.loc[idx, 'p'] = \
            old_coefs_and_pvalues[old_coefs_and_pvalues['covariate']==var_]['p'].iloc[0]

models_with_p_below_0_2_df.\
to_csv('data/'+date_dir+'/exports/'+model_type+'/Dec_10th_models_with_p_below_0_2_for_had_online_order_last_month.csv',\
      index = False)

curr_model_names = pd.read_csv('combinations_of_variables_that_are_not_dependent/model_names.csv')
curr_combinations = pd.read_csv('combinations_of_variables_that_are_not_dependent/combinations_of_variables.csv')
model_num = get_model_number(var_=base_var_, model_names=curr_model_names)

datasets = ['ALL_spots_with_CB_cancellation_confirmed', 'ALL_spots_with_CB_cancellation_requested',\
                     'ALL_spots_wo_CB_cancellation_confirmed', 'ALL_spots_wo_CB_cancellation_requested',\
                     'CAN_CANCEL_spots_with_CB_cancellation_confirmed', 'CAN_CANCEL_spots_with_CB_cancellation_requested',\
                     'CAN_CANCEL_spots_wo_CB_cancellation_confirmed', 'CAN_CANCEL_spots_wo_CB_cancellation_requested']

model_9_coefs_and_pvalues = \
pd.DataFrame(columns = ['model_number', 'model_name', 'data_set', 'variable', 'condition', 'coef', 'exp(coef)', 'p'])
conditions_described = {'no_condition':'with all initial variables',\
                            'first_condition':'without categorical variables with |coeff| < 0.2 and p value > 0.2; without numerical variables with |coeff| < 0.01 and p value > 0.2',\
                            'second_condition':'without variables p value >= 0.5',\
                            'third_condition':'without variables p value >= 0.2'}

# for file in ['coef_and_pvalues_'+x+'.csv' for x in datasets]:
for file in ['with_visited_inquiries_page_coef_and_pvalues_'+x+'.csv' for x in datasets]:
    for condition in conditions_described.keys():
        coefs_and_pvalues = \
        pd.read_csv('data/'+date_dir+'/exports/coefficients_and_pvalues/'+model_type+'/'+var_+'/'+condition+'/'+file)
        idx = len(model_9_coefs_and_pvalues)
        if var_ in coefs_and_pvalues['covariate'].unique():
            model_9_coefs_and_pvalues.loc[idx, 'model_number'] = model_num
            model_9_coefs_and_pvalues.loc[idx, 'model_name'] = \
            curr_model_names[curr_model_names['model_number']==model_num]['model_name'].iloc[0]
            model_9_coefs_and_pvalues.loc[idx, 'data_set'] = file.split('.csv')[0]
            model_9_coefs_and_pvalues.loc[idx, 'variable'] = var_
            model_9_coefs_and_pvalues.loc[idx, 'condition'] = conditions_described[condition]
            model_9_coefs_and_pvalues.loc[idx, 'coef'] = \
            coefs_and_pvalues[coefs_and_pvalues['covariate']==var_]['coef'].iloc[0]
            model_9_coefs_and_pvalues.loc[idx, 'exp(coef)'] = \
            coefs_and_pvalues[coefs_and_pvalues['covariate']==var_]['exp(coef)'].iloc[0]
            model_9_coefs_and_pvalues.loc[idx, 'p'] = \
            coefs_and_pvalues[coefs_and_pvalues['covariate']==var_]['p'].iloc[0] 
        
model_9_coefs_and_pvalues.\
to_csv('data/'+date_dir+'/exports/'+model_type+'/model_with_visited_inquiries_page_and_online_order_results.csv')

date of analysis: 2021-12-01
Model type: started_doing_something
